In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import f_oneway, ttest_ind, norm
import warnings
warnings.filterwarnings('ignore')

In [3]:
print("="*80)
print("PLACEMENT DATASET ANALYSIS WITH DETAILED EXPLANATIONS")
print("="*80)

PLACEMENT DATASET ANALYSIS WITH DETAILED EXPLANATIONS


In [4]:
df = pd.read_csv('Placement.csv')
print("\nDataset Overview:")
print(f"Total Records: {len(df)}")
print(f"Total Columns: {len(df.columns)}")
print(f"\nColumn Names and Types:")
print(df.dtypes)
print("\n" + "="*80)
print("STEP 1: CHECKING FOR MISSING VALUES (NaN)")
print("="*80)


Dataset Overview:
Total Records: 215
Total Columns: 15

Column Names and Types:
sl_no               int64
gender             object
ssc_p             float64
ssc_b              object
hsc_p             float64
hsc_b              object
hsc_s              object
degree_p          float64
degree_t           object
workex             object
etest_p           float64
specialisation     object
mba_p             float64
status             object
salary            float64
dtype: object

STEP 1: CHECKING FOR MISSING VALUES (NaN)


In [5]:
print("\nMissing Values Count:")
missing_values = df.isnull().sum()
print(missing_values)

print("\nMissing Values Percentage:")
missing_percentage = (df.isnull().sum() / len(df)) * 100
missing_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing_Count': missing_values.values,
    'Missing_Percentage': missing_percentage.values
})
print(missing_df[missing_df['Missing_Count'] > 0])

print("\n" + "-"*80)
print("EXPLANATION: HANDLING MISSING VALUES")
print("-"*80)


Missing Values Count:
sl_no              0
gender             0
ssc_p              0
ssc_b              0
hsc_p              0
hsc_b              0
hsc_s              0
degree_p           0
degree_t           0
workex             0
etest_p            0
specialisation     0
mba_p              0
status             0
salary            67
dtype: int64

Missing Values Percentage:
    Column  Missing_Count  Missing_Percentage
14  salary             67           31.162791

--------------------------------------------------------------------------------
EXPLANATION: HANDLING MISSING VALUES
--------------------------------------------------------------------------------


In [6]:
print("\nChecking for unexpected NaN values:")
for col in df.columns:
    if col != 'salary' and df[col].isnull().sum() > 0:
        print(f"WARNING: {col} has {df[col].isnull().sum()} missing values")
        
        # Suggest fixing method based on column type
        if df[col].dtype in ['float64', 'int64']:
            print(f"  → FIX: Fill with median: df['{col}'].fillna(df['{col}'].median(), inplace=True)")
        else:
            print(f"  → FIX: Fill with mode: df['{col}'].fillna(df['{col}'].mode()[0], inplace=True)")

# Create a clean copy for salary analysis
df_placed = df[df['status'] == 'Placed'].copy()
df_not_placed = df[df['status'] == 'Not Placed'].copy()

print(f"\nAfter Separating:")
print(f"Placed Students: {len(df_placed)}")
print(f"Not Placed Students: {len(df_not_placed)}")


Checking for unexpected NaN values:

After Separating:
Placed Students: 148
Not Placed Students: 67


In [7]:
print("\n" + "="*80)
print("QUESTION 1: HOW MANY STUDENTS ARE NOT PLACED?")
print("="*80)

placement_counts = df['status'].value_counts()
not_placed_count = placement_counts.get('Not Placed', 0)
placed_count = placement_counts.get('Placed', 0)

print(f"\nPlacement Status:")
print(f"✓ Placed: {placed_count} students ({placed_count/len(df)*100:.2f}%)")
print(f"✗ Not Placed: {not_placed_count} students ({not_placed_count/len(df)*100:.2f}%)")

print("\n" + "-"*80)
print("EXPLANATION:")
print("-"*80)
print(f"""
ANSWER: {not_placed_count} students are NOT PLACED

INTERPRETATION:
- Out of {len(df)} total students, {not_placed_count} did not get placement
- This represents {not_placed_count/len(df)*100:.2f}% of the entire batch
- Placement rate is {placed_count/len(df)*100:.2f}%

CODE USED:
df['status'].value_counts()  # Counts occurrences of each status
""")


QUESTION 1: HOW MANY STUDENTS ARE NOT PLACED?

Placement Status:
✓ Placed: 148 students (68.84%)
✗ Not Placed: 67 students (31.16%)

--------------------------------------------------------------------------------
EXPLANATION:
--------------------------------------------------------------------------------

ANSWER: 67 students are NOT PLACED

INTERPRETATION:
- Out of 215 total students, 67 did not get placement
- This represents 31.16% of the entire batch
- Placement rate is 68.84%

CODE USED:
df['status'].value_counts()  # Counts occurrences of each status



In [9]:
print("\n" + "="*80)
print("QUESTION 2: FIND REASONS FOR NON-PLACEMENT")
print("="*80)

print("\nComparing Academic Performance:")
print("-"*60)

comparison_cols = ['ssc_p', 'hsc_p', 'degree_p', 'etest_p', 'mba_p']
comparison_results = []

for col in comparison_cols:
    placed_mean = df_placed[col].mean()
    not_placed_mean = df_not_placed[col].mean()
    diff = placed_mean - not_placed_mean
    
    comparison_results.append({
        'Metric': col,
        'Placed_Mean': placed_mean,
        'NotPlaced_Mean': not_placed_mean,
        'Difference': diff
    })
    
    print(f"{col:12s}: Placed={placed_mean:6.2f}%, Not Placed={not_placed_mean:6.2f}%, Diff={diff:+6.2f}%")

print("\nWork Experience Analysis:")
print("-"*60)
workex_placed = df_placed['workex'].value_counts()
workex_not_placed = df_not_placed['workex'].value_counts()

placed_with_exp = workex_placed.get('Yes', 0) / len(df_placed) * 100
notplaced_with_exp = workex_not_placed.get('Yes', 0) / len(df_not_placed) * 100

print(f"Placed students with Work Experience: {placed_with_exp:.2f}%")
print(f"Not Placed students with Work Experience: {notplaced_with_exp:.2f}%")

print("\nGender Distribution:")
print("-"*60)
gender_placed = df_placed['gender'].value_counts()
gender_not_placed = df_not_placed['gender'].value_counts()

for gender in df['gender'].unique():
    placed_pct = gender_placed.get(gender, 0) / len(df_placed) * 100
    notplaced_pct = gender_not_placed.get(gender, 0) / len(df_not_placed) * 100
    print(f"Gender '{gender}': Placed={placed_pct:.2f}%, Not Placed={notplaced_pct:.2f}%")

print("\n" + "-"*80)
print("EXPLANATION:")
print("-"*80)


QUESTION 2: FIND REASONS FOR NON-PLACEMENT

Comparing Academic Performance:
------------------------------------------------------------
ssc_p       : Placed= 71.72%, Not Placed= 57.54%, Diff=+14.18%
hsc_p       : Placed= 69.93%, Not Placed= 58.40%, Diff=+11.53%
degree_p    : Placed= 68.74%, Not Placed= 61.13%, Diff= +7.61%
etest_p     : Placed= 73.24%, Not Placed= 69.59%, Diff= +3.65%
mba_p       : Placed= 62.58%, Not Placed= 61.61%, Diff= +0.97%

Work Experience Analysis:
------------------------------------------------------------
Placed students with Work Experience: 43.24%
Not Placed students with Work Experience: 14.93%

Gender Distribution:
------------------------------------------------------------
Gender 'M': Placed=67.57%, Not Placed=58.21%
Gender 'F': Placed=32.43%, Not Placed=41.79%

--------------------------------------------------------------------------------
EXPLANATION:
--------------------------------------------------------------------------------


In [9]:
from scipy.stats import pearsonr
correlation, p_value = pearsonr(df_placed['mba_p'], df_placed['salary'])

print(f"\nPearson Correlation Coefficient: {correlation:.4f}")
print(f"P-value: {p_value:.6f}")

# Interpret strength
if abs(correlation) > 0.7:
    strength = "STRONG"
elif abs(correlation) > 0.4:
    strength = "MODERATE"
elif abs(correlation) > 0.2:
    strength = "WEAK"
else:
    strength = "VERY WEAK"

direction = "POSITIVE" if correlation > 0 else "NEGATIVE"

print(f"\nInterpretation: {strength} {direction} correlation")

if p_value < 0.05:
    print("Statistical Significance: YES (p < 0.05) - Result is statistically significant")
else:
    print("Statistical Significance: NO (p >= 0.05) - Result is NOT statistically significant")

print("\n" + "-"*80)
print("EXPLANATION:")
print("-"*80)


Pearson Correlation Coefficient: 0.1750
P-value: 0.033377

Interpretation: VERY WEAK POSITIVE correlation
Statistical Significance: YES (p < 0.05) - Result is statistically significant

--------------------------------------------------------------------------------
EXPLANATION:
--------------------------------------------------------------------------------


In [10]:
print("\n" + "="*80)
print("QUESTION 4: WHICH SPECIALIZATION IS GETTING MINIMUM SALARY?")
print("="*80)

salary_by_spec = df_placed.groupby('specialisation')['salary'].agg([
    'count', 'mean', 'median', 'min', 'max', 'std'
])

print("\nSalary Statistics by Specialization:")
print(salary_by_spec)

min_spec = salary_by_spec['mean'].idxmin()
min_salary = salary_by_spec.loc[min_spec, 'mean']

print(f"\n{'='*60}")
print(f"ANSWER: {min_spec}")
print(f"Average Salary: ₹{min_salary:,.2f}")
print(f"{'='*60}")

print("\n" + "-"*80)
print("EXPLANATION:")
print("-"*80)


QUESTION 4: WHICH SPECIALIZATION IS GETTING MINIMUM SALARY?

Salary Statistics by Specialization:
                count           mean    median       min       max  \
specialisation                                                       
Mkt&Fin            95  298852.631579  270000.0  200000.0  940000.0   
Mkt&HR             53  270377.358491  255000.0  200000.0  450000.0   

                          std  
specialisation                 
Mkt&Fin         108190.530165  
Mkt&HR           54784.271471  

ANSWER: Mkt&HR
Average Salary: ₹270,377.36

--------------------------------------------------------------------------------
EXPLANATION:
--------------------------------------------------------------------------------


In [15]:
print("\n" + "="*80)
print("QUESTION 5: HOW MANY STUDENTS ARE GETTING ABOVE ₹5,00,000 SALARY?")
print("="*80)

above_5L = df_placed[df_placed['salary'] > 500000]
count_above_5L = len(above_5L)
percent_above_5L = (count_above_5L / len(df_placed)) * 100

print(f"\nStudents earning > ₹5,00,000: {count_above_5L} out of {len(df_placed)}")
print(f"Percentage: {percent_above_5L:.2f}%")

# Additional salary ranges
print("\nSalary Distribution by Ranges:")
print("-"*60)
ranges = [
    (0, 300000, "Below 3 Lakhs"),
    (300000, 500000, "3-5 Lakhs"),
    (500000, 700000, "5-7 Lakhs"),
    (700000, 1000000, "Above 7 Lakhs")
]

for low, high, label in ranges:
    count = len(df_placed[(df_placed['salary'] > low) & (df_placed['salary'] <= high)])
    pct = (count / len(df_placed)) * 100
    print(f"{label:20s}: {count:3d} students ({pct:5.2f}%)")

print("\n" + "-"*80)
print("EXPLANATION:")
print("-"*80)


QUESTION 5: HOW MANY STUDENTS ARE GETTING ABOVE ₹5,00,000 SALARY?

Students earning > ₹5,00,000: 3 out of 148
Percentage: 2.03%

Salary Distribution by Ranges:
------------------------------------------------------------
Below 3 Lakhs       : 120 students (81.08%)
3-5 Lakhs           :  25 students (16.89%)
5-7 Lakhs           :   2 students ( 1.35%)
Above 7 Lakhs       :   1 students ( 0.68%)

--------------------------------------------------------------------------------
EXPLANATION:
--------------------------------------------------------------------------------


In [15]:
print("\n" + "="*80)
print("QUESTION 7: ANOVA TEST - VARIANCE BETWEEN etest_p AND mba_p (α = 0.05)")
print("="*80)


QUESTION 7: ANOVA TEST - VARIANCE BETWEEN etest_p AND mba_p (α = 0.05)


In [16]:
var_etest = df['etest_p'].var()
var_mba = df['mba_p'].var()

In [17]:
f_statistic = var_etest / var_mba if var_etest > var_mba else var_mba / var_etest
df1 = len(df) - 1
df2 = len(df) - 1

p_value_f = 1 - stats.f.cdf(f_statistic, df1, df2)

print("\nHypothesis Testing Setup:")
print("-"*60)
print("H₀ (Null Hypothesis): No significant difference in variance")
print("H₁ (Alternative Hypothesis): Significant difference in variance exists")
print(f"Significance Level: α = 0.05")

print("\nVariance Analysis:")
print("-"*60)
print(f"Variance of etest_p: {var_etest:.4f}")
print(f"Variance of mba_p: {var_mba:.4f}")
print(f"F-statistic: {f_statistic:.4f}")
print(f"P-value: {p_value_f:.6f}")

print("\n" + "="*60)
if p_value_f < 0.05:
    print("DECISION: REJECT H₀")
    print("CONCLUSION: Significant difference in variance EXISTS")
    print("Interpretation: etest_p and mba_p have different variability")
else:
    print("DECISION: FAIL TO REJECT H₀")
    print("CONCLUSION: No significant difference in variance")
    print("Interpretation: etest_p and mba_p have similar variability")
print("="*60)

print("\n" + "-"*80)
print("EXPLANATION:")
print("-"*80)


Hypothesis Testing Setup:
------------------------------------------------------------
H₀ (Null Hypothesis): No significant difference in variance
H₁ (Alternative Hypothesis): Significant difference in variance exists
Significance Level: α = 0.05

Variance Analysis:
------------------------------------------------------------
Variance of etest_p: 176.2510
Variance of mba_p: 34.0284
F-statistic: 5.1795
P-value: 0.000000

DECISION: REJECT H₀
CONCLUSION: Significant difference in variance EXISTS
Interpretation: etest_p and mba_p have different variability

--------------------------------------------------------------------------------
EXPLANATION:
--------------------------------------------------------------------------------


In [18]:
print("\n" + "="*80)
print("QUESTION 8: T-TEST - SIMILARITY BETWEEN GROUPS W.R.T SALARY (α = 0.05)")
print("="*80)


QUESTION 8: T-TEST - SIMILARITY BETWEEN GROUPS W.R.T SALARY (α = 0.05)


In [16]:
print("\n--- TEST 1: Degree Type (Sci&Tech vs Others) ---")
print("-"*60)

sci_tech_sal = df_placed[df_placed['degree_t'] == 'Sci&Tech']['salary']
others_sal = df_placed[df_placed['degree_t'] != 'Sci&Tech']['salary']

print("H₀: Mean salary is SAME for both degree types")
print("H₁: Mean salary is DIFFERENT for both degree types")

if len(sci_tech_sal) > 1 and len(others_sal) > 1:
    t_stat_deg, p_val_deg = ttest_ind(sci_tech_sal, others_sal)
    
    print(f"\nSample Sizes:")
    print(f"  Sci&Tech: n = {len(sci_tech_sal)}")
    print(f"  Others: n = {len(others_sal)}")
    
    print(f"\nMean Salaries:")
    print(f"  Sci&Tech: ₹{sci_tech_sal.mean():,.2f}")
    print(f"  Others: ₹{others_sal.mean():,.2f}")
    print(f"  Difference: ₹{abs(sci_tech_sal.mean() - others_sal.mean()):,.2f}")
    
    print(f"\nTest Results:")
    print(f"  T-statistic: {t_stat_deg:.4f}")
    print(f"  P-value: {p_val_deg:.6f}")
    
    print("\n" + "="*60)
    if p_val_deg < 0.05:
        print("DECISION: REJECT H₀")
        print("CONCLUSION: Mean salaries are SIGNIFICANTLY DIFFERENT")
    else:
        print("DECISION: FAIL TO REJECT H₀")
        print("CONCLUSION: Mean salaries are SIMILAR (no significant difference)")
    print("="*60)


--- TEST 1: Degree Type (Sci&Tech vs Others) ---
------------------------------------------------------------
H₀: Mean salary is SAME for both degree types
H₁: Mean salary is DIFFERENT for both degree types

Sample Sizes:
  Sci&Tech: n = 41
  Others: n = 107

Mean Salaries:
  Sci&Tech: ₹314,609.76
  Others: ₹278,710.28
  Difference: ₹35,899.48

Test Results:
  T-statistic: 2.1159
  P-value: 0.036047

DECISION: REJECT H₀
CONCLUSION: Mean salaries are SIGNIFICANTLY DIFFERENT


In [17]:
print("\n--- TEST 2: Specialization (Mkt&HR vs Mkt&Fin) ---")
print("-"*60)

mkt_hr_sal = df_placed[df_placed['specialisation'] == 'Mkt&HR']['salary']
mkt_fin_sal = df_placed[df_placed['specialisation'] == 'Mkt&Fin']['salary']

print("H₀: Mean salary is SAME for both specializations")
print("H₁: Mean salary is DIFFERENT for both specializations")

if len(mkt_hr_sal) > 1 and len(mkt_fin_sal) > 1:
    t_stat_spec, p_val_spec = ttest_ind(mkt_hr_sal, mkt_fin_sal)
    
    print(f"\nSample Sizes:")
    print(f"  Mkt&HR: n = {len(mkt_hr_sal)}")
    print(f"  Mkt&Fin: n = {len(mkt_fin_sal)}")
    
    print(f"\nMean Salaries:")
    print(f"  Mkt&HR: ₹{mkt_hr_sal.mean():,.2f}")
    print(f"  Mkt&Fin: ₹{mkt_fin_sal.mean():,.2f}")
    print(f"  Difference: ₹{abs(mkt_hr_sal.mean() - mkt_fin_sal.mean()):,.2f}")
    
    print(f"\nTest Results:")
    print(f"  T-statistic: {t_stat_spec:.4f}")
    print(f"  P-value: {p_val_spec:.6f}")
    
    print("\n" + "="*60)
    if p_val_spec < 0.05:
        print("DECISION: REJECT H₀")
        print("CONCLUSION: Mean salaries are SIGNIFICANTLY DIFFERENT")
    else:
        print("DECISION: FAIL TO REJECT H₀")
        print("CONCLUSION: Mean salaries are SIMILAR (no significant difference)")
    print("="*60)

print("\n" + "-"*80)
print("EXPLANATION:")
print("-"*80)


--- TEST 2: Specialization (Mkt&HR vs Mkt&Fin) ---
------------------------------------------------------------
H₀: Mean salary is SAME for both specializations
H₁: Mean salary is DIFFERENT for both specializations

Sample Sizes:
  Mkt&HR: n = 53
  Mkt&Fin: n = 95

Mean Salaries:
  Mkt&HR: ₹270,377.36
  Mkt&Fin: ₹298,852.63
  Difference: ₹28,475.27

Test Results:
  T-statistic: -1.7904
  P-value: 0.075457

DECISION: FAIL TO REJECT H₀
CONCLUSION: Mean salaries are SIMILAR (no significant difference)

--------------------------------------------------------------------------------
EXPLANATION:
--------------------------------------------------------------------------------


In [21]:
print("\n" + "="*80)
print("QUESTION 9: CONVERT TO STANDARD NORMAL DISTRIBUTION (Z-SCORES)")
print("="*80)


QUESTION 9: CONVERT TO STANDARD NORMAL DISTRIBUTION (Z-SCORES)


In [18]:
salary_mean = df_placed['salary'].mean()
salary_std = df_placed['salary'].std()


In [20]:
df_placed['salary_z_score'] = (df_placed['salary'] - salary_mean) / salary_std

print("\nOriginal Distribution (Salary):")
print("-"*60)
print(f"Mean (μ): ₹{salary_mean:,.2f}")
print(f"Standard Deviation (σ): ₹{salary_std:,.2f}")
print(f"Min: ₹{df_placed['salary'].min():,.2f}")
print(f"Max: ₹{df_placed['salary'].max():,.2f}")

print("\nStandardized Distribution (Z-scores):")
print("-"*60)
print(f"Mean: {df_placed['salary_z_score'].mean():.10f} (should be ≈ 0)")
print(f"Standard Deviation: {df_placed['salary_z_score'].std():.10f} (should be ≈ 1)")
print(f"Min Z-score: {df_placed['salary_z_score'].min():.4f}")
print(f"Max Z-score: {df_placed['salary_z_score'].max():.4f}")

print("\nSample Z-score Conversions:")
print("-"*60)
sample_data = df_placed[['salary', 'salary_z_score']].head(10)
for idx, row in sample_data.iterrows():
    print(f"Salary: ₹{row['salary']:>10,.2f} → Z-score: {row['salary_z_score']:>7.4f}")

print("\n" + "-"*80)
print("EXPLANATION:")
print("-"*80)


Original Distribution (Salary):
------------------------------------------------------------
Mean (μ): ₹288,655.41
Standard Deviation (σ): ₹93,457.45
Min: ₹200,000.00
Max: ₹940,000.00

Standardized Distribution (Z-scores):
------------------------------------------------------------
Mean: 0.0000000000 (should be ≈ 0)
Standard Deviation: 1.0000000000 (should be ≈ 1)
Min Z-score: -0.9486
Max Z-score: 6.9694

Sample Z-score Conversions:
------------------------------------------------------------
Salary: ₹270,000.00 → Z-score: -0.1996
Salary: ₹200,000.00 → Z-score: -0.9486
Salary: ₹250,000.00 → Z-score: -0.4136
Salary: ₹425,000.00 → Z-score:  1.4589
Salary: ₹252,000.00 → Z-score: -0.3922
Salary: ₹231,000.00 → Z-score: -0.6169
Salary: ₹260,000.00 → Z-score: -0.3066
Salary: ₹250,000.00 → Z-score: -0.4136
Salary: ₹218,000.00 → Z-score: -0.7560
Salary: ₹200,000.00 → Z-score: -0.9486

--------------------------------------------------------------------------------
EXPLANATION:
---------------

In [21]:
print("\n" + "="*80)
print("QUESTION 10: PROBABILITY DENSITY FUNCTION - SALARY ₹7L to ₹9L")
print("="*80)

# Calculate Z-scores for the range
z_low = (700000 - salary_mean) / salary_std
z_high = (900000 - salary_mean) / salary_std



QUESTION 10: PROBABILITY DENSITY FUNCTION - SALARY ₹7L to ₹9L


In [22]:
z_low = (700000 - salary_mean) / salary_std
z_high = (900000 - salary_mean) / salary_std

In [23]:
prob_below_900k = norm.cdf(z_high)
prob_below_700k = norm.cdf(z_low)
probability = prob_below_900k - prob_below_700k

print("\nAssuming Normal Distribution:")
print("-"*60)
print(f"Mean: ₹{salary_mean:,.2f}")
print(f"Standard Deviation: ₹{salary_std:,.2f}")

print("\nZ-score Conversion:")
print("-"*60)
print(f"₹7,00,000 → Z-score: {z_low:.4f}")
print(f"₹9,00,000 → Z-score: {z_high:.4f}")

print("\nProbability Calculation:")
print("-"*60)
print(f"P(Salary < ₹9,00,000) = {prob_below_900k:.4f}")
print(f"P(Salary < ₹7,00,000) = {prob_below_700k:.4f}")
print(f"P(₹7L < Salary < ₹9L) = {probability:.4f} ({probability*100:.2f}%)")


Assuming Normal Distribution:
------------------------------------------------------------
Mean: ₹288,655.41
Standard Deviation: ₹93,457.45

Z-score Conversion:
------------------------------------------------------------
₹7,00,000 → Z-score: 4.4014
₹9,00,000 → Z-score: 6.5414

Probability Calculation:
------------------------------------------------------------
P(Salary < ₹9,00,000) = 1.0000
P(Salary < ₹7,00,000) = 1.0000
P(₹7L < Salary < ₹9L) = 0.0000 (0.00%)


In [24]:
actual_in_range = len(df_placed[(df_placed['salary'] >= 700000) & 
                                 (df_placed['salary'] <= 900000)])
actual_probability = actual_in_range / len(df_placed)

print("\nActual Data Verification:")
print("-"*60)
print(f"Students in range: {actual_in_range} out of {len(df_placed)}")
print(f"Actual probability: {actual_probability:.4f} ({actual_probability*100:.2f}%)")
print(f"Difference: {abs(probability - actual_probability):.4f} ({abs(probability - actual_probability)*100:.2f}%)")

print("\n" + "-"*80)
print("EXPLANATION:")
print("-"*80)


Actual Data Verification:
------------------------------------------------------------
Students in range: 0 out of 148
Actual probability: 0.0000 (0.00%)
Difference: 0.0000 (0.00%)

--------------------------------------------------------------------------------
EXPLANATION:
--------------------------------------------------------------------------------
